# 超參數及集成學習權重最佳化

我們的目的是要以前一天的氣象預報來預測當天的電力資訊，但是我們手上的氣象預報歷史資料是從2024年七月開始蒐集，直接拿來預測電力資訊天數不夠，  
所以除了 Power_predict.ipynb 裡面敘述的從氣象觀測歷史資料預測電力資料的模型之外，我們還需要建立從氣象預報資料來預測氣象觀測資料的模型。  

這裡的氣象資料預測的模型建立方式跟電力資料的模型大同小異，主要的不同點是氣象資料預測可以把每天每站的數據當成一個樣本，這樣我們就可以在相對短時間之內累積足夠的樣本數。

同時這個筆記本也要處理超參數的最佳化，我們使用 optuna 這個第三方套件來達成這個任務。  
另外我們也嘗試最佳化集成學習時各模型的權重，具體方法為計算出模型之間的誤差相關矩陣，再從這個矩陣解出最佳權重組合。

整個預測系統在真實世界資料中運行的情形可以到<a href='http://ec2-54-206-30-159.ap-southeast-2.compute.amazonaws.com:8501'> 這個網站 </a> 查看

## 初始化

### 匯入模組與套件

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
#這兩行讓 matplotlib 的圖可以顯示中文，同時正常顯示負號
matplotlib.rc('font', family='Microsoft JhengHei')
plt.rcParams['axes.unicode_minus'] = False
import datetime
from copy import deepcopy
import os
import joblib
import json
from tqdm import tqdm
import optuna

# 設置Optuna日誌級別為 WARNING，僅顯示警告及以上級別的信息
optuna.logging.set_verbosity(optuna.logging.WARNING)

pd.set_option('future.no_silent_downcasting', True)

In [2]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.svm import SVR, NuSVR

In [3]:
from sklearn.svm import SVC, NuSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [7]:
from Pytorch_models.metrics import Array_Metrics
from Pytorch_models import models as pytorch_models
from Pytorch_models import api
MAE = Array_Metrics.mae
R2_score = Array_Metrics.r2

In [8]:
from utils.prepare_data import prepare_forecast_observation_df, prepare_data

In [9]:
def FCN_model(input_f, output_f, feature_counts, dropout_factor=0, L2_factor=1e-15, mode='regressor'):
    if mode == 'regressor':
        model = pytorch_models.SimpleNN(input_f, output_f, feature_counts, dropout_factor)
    elif mode == 'classifier':
        model = pytorch_models.SimpleNN_classifier(input_f, output_f, feature_counts, dropout_factor)
    Model_API = api.Model_API(model, L2_factor=L2_factor, classifier=(mode=='classifier'))
    return Model_API

### 初始參數

In [10]:
data_path = './historical/data/'

# 資料的開始與結束日期
start_date = '2023-01-01'
end_date = '2024-10-15'

train_model_path = f'./trained_model_parameters/model_meta_{end_date}/'

In [11]:
# 設定要不要開啟快速測試模式
speed_test = False

#---------------------------------------------------------------------------------
# 設定要不要重算所有超參數與權重，或是不要做任何重算
# 如果兩者皆為 False 則按照下面的個別設定
# 如果兩者皆為 True 則全部重算
#
# Note: 如果指定的路徑當中沒有相應的 meta.json 檔，程式將會無視這邊的設定而進行計算
#---------------------------------------------------------------------------------
rerun_all_calculation = False
dont_run_any_calculation = False


# 此值為 False 則重新計算，True 則從存檔中讀取
optuna_has_done = {
    '日照率': True,
    '最高氣溫': True,
    '最低氣溫': True,
    '氣溫': True,
    '風速': True,
    '風力': False,
    '太陽能': False,
    '尖峰負載': False,
    '夜尖峰': True,
    '午後平均風速': True,
    '午後平均氣溫': True,
    '下午平均風速': True,
    '下午平均氣溫': True,
    '傍晚平均風速': True,
    '傍晚平均氣溫': True,
}

weights_has_determined = {
    '日照率': True,
    '最高氣溫': True,
    '最低氣溫': True,
    '氣溫': True,
    '風速': True,
    '風力': False,
    '太陽能': False,
    '尖峰負載': False,
    '夜尖峰': True,
    '午後平均風速': True,
    '午後平均氣溫': True,
    '下午平均風速': True,
    '下午平均氣溫': True,
    '傍晚平均風速': True,
    '傍晚平均氣溫': True,
}

# 如果前面有設定全部重算或都不要重算，則重設上面的這兩個字典
if dont_run_any_calculation:
    optuna_has_done = {k: True for k in optuna_has_done.keys()}
    weights_has_determined = {k: True for k in weights_has_determined.keys()}

if rerun_all_calculation:
    optuna_has_done = {k: False for k in optuna_has_done.keys()}
    weights_has_determined = {k: False for k in weights_has_determined.keys()}

In [12]:
# 定義每個 model_label 對應的 model
model_class_dict = {}
model_class_dict['regressor'] = {
    'LinearRegression': LinearRegression,
    'RandomForest': RandomForestRegressor,
    'XGBoost': XGBRegressor,
    'LightGBM': LGBMRegressor,
    'SVR': SVR,
    'NuSVR': NuSVR,
    'FCN': FCN_model,
}
model_class_dict['classifier'] = {
    'RandomForest': RandomForestClassifier,
    'XGBoost': XGBClassifier,
    'LightGBM': LGBMClassifier,
    'SVC': SVC,
    'NuSVC': NuSVC,
    'LogisticRegression': LogisticRegression,
    'FCN': FCN_model,
}

### 讀取資料

讀取先前經由爬蟲定時抓取的預報與觀測資料

In [13]:
forecast_obs_df = prepare_forecast_observation_df(data_path, start_date=start_date, end_date=end_date)
weather_power_df = prepare_data(data_path, start_date=start_date, end_date=end_date)

In [14]:
list(forecast_obs_df.columns)

['鄉鎮',
 '日期',
 '晴預報_0',
 '多雲預報_0',
 '陰預報_0',
 '短暫陣雨預報_0',
 '短暫陣雨或雷雨預報_0',
 '午後短暫雷陣雨預報_0',
 '陣雨或雷雨預報_0',
 '溫度預報_0',
 '降水機率預報_0',
 '相對溼度預報_0',
 '風速預報_0',
 '晴預報_3',
 '多雲預報_3',
 '陰預報_3',
 '短暫陣雨預報_3',
 '短暫陣雨或雷雨預報_3',
 '午後短暫雷陣雨預報_3',
 '陣雨或雷雨預報_3',
 '溫度預報_3',
 '降水機率預報_3',
 '相對溼度預報_3',
 '風速預報_3',
 '晴預報_6',
 '多雲預報_6',
 '陰預報_6',
 '短暫陣雨預報_6',
 '短暫陣雨或雷雨預報_6',
 '午後短暫雷陣雨預報_6',
 '陣雨或雷雨預報_6',
 '溫度預報_6',
 '降水機率預報_6',
 '相對溼度預報_6',
 '風速預報_6',
 '晴預報_9',
 '多雲預報_9',
 '陰預報_9',
 '短暫陣雨預報_9',
 '短暫陣雨或雷雨預報_9',
 '午後短暫雷陣雨預報_9',
 '陣雨或雷雨預報_9',
 '溫度預報_9',
 '降水機率預報_9',
 '相對溼度預報_9',
 '風速預報_9',
 '晴預報_12',
 '多雲預報_12',
 '陰預報_12',
 '短暫陣雨預報_12',
 '短暫陣雨或雷雨預報_12',
 '午後短暫雷陣雨預報_12',
 '陣雨或雷雨預報_12',
 '溫度預報_12',
 '降水機率預報_12',
 '相對溼度預報_12',
 '風速預報_12',
 '晴預報_15',
 '多雲預報_15',
 '陰預報_15',
 '短暫陣雨預報_15',
 '短暫陣雨或雷雨預報_15',
 '午後短暫雷陣雨預報_15',
 '陣雨或雷雨預報_15',
 '溫度預報_15',
 '降水機率預報_15',
 '相對溼度預報_15',
 '風速預報_15',
 '晴預報_18',
 '多雲預報_18',
 '陰預報_18',
 '短暫陣雨預報_18',
 '短暫陣雨或雷雨預報_18',
 '午後短暫雷陣雨預報_18',
 '陣雨或雷雨預報_18',
 '溫度預報_18',
 '降水機率預報_18',
 '相對溼度預報_18',

## 函數

### 超參數最佳化

這部分的函數有：  
1. get_XY: 從 DataFrame 中提取需要的 X 與 Y 兩個 numpy array。
2. five_fold_test: 執行一次 5-fold 測試，會呼叫 get_XY_from_forecast_and_observation。
3. assign_model: 根據 model_label 與超參數字典建立一個模型。
3. hyperparameter_tuning: 針對特定的模型與超參數組合，呼叫 five_fold_test 執行多次 5-fold 測試，並回傳 R2 值。
4. optuna_operation: 利用第三方套件 optuna 執行超參數調整，會呼叫 hyperparameter_tuning。

流程控制函數 flow_control 會呼叫 optuna_operation，而主程式只會直接呼叫 flow_control。

In [15]:
def get_XY(data_df, Y_feature, X_features=None, hours=[str(i) for i in range(0, 24, 3)]):
    date_related_cols = ['日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '白日長度']
    
    if Y_feature in ['最高氣溫', '最低氣溫', '氣溫', '風速', '日照率', '全天空日射量']:
        target = 'obs'
    elif '平均' in Y_feature:
        target = 'obs'
    elif Y_feature in ['風力', '太陽能', '尖峰負載', '夜尖峰']:
        target = 'pwd'

    X_cols = []
    if X_features is None:
        for this_col in data_df.columns:
            if '_' in this_col:
                X_cols.append(this_col)
        if target == 'pwd':
            X_cols += date_related_cols
    else:
        for col in data_df.columns:
            #print(col)
            if target == 'obs':
                dash_splited = col.split('預報_')
            elif target == 'pwd':
                dash_splited = col.split('_')
            #print(dash_splited)
            if len(dash_splited) >= 2:
                #if dash_splited[0] in X_features and (target=='pwd' or dash_splited[1] in hours):
                if dash_splited[0] in X_features:
                    X_cols.append(col)
            else:
                if col in date_related_cols and col in X_features:
                    X_cols.append(col)

    Xs = np.array(data_df[X_cols])
    Ys = np.array(data_df[Y_feature])

    Xs = Xs[np.invert(np.isnan(Ys)),:]
    Ys = Ys[np.invert(np.isnan(Ys))]

    return Xs, Ys, X_cols

In [16]:
def five_fold_test(Xs, Ys, model=XGBRegressor(), mode='regressor',
                   deep_learning=False, fold_n=5, standard_scale=True, always_test_last_chunk=False, fit_square=False):
    
    def metric(Y_test, Y_pred, mode=mode):
        if mode == 'regressor':
            return 1 - np.mean((Y_test - Y_pred)**2) / np.var(Y_test)
        elif mode == 'classifier':
            return f1_score(Y_test, Y_pred)

    shuffle = not always_test_last_chunk
    kf = KFold(n_splits=fold_n, shuffle=shuffle)
    
    XY_folds = {}
    for i, (train_index, test_index) in enumerate(kf.split(Xs)):
        XY_folds[i] = (train_index, test_index)
    
    metric_test_list, metric_train_list = [], []

    if always_test_last_chunk:
        iters = [fold_n-1]
    else:
        iters = range(fold_n)
    
    for i in iters:
        if deep_learning:
            input_f = model.model.params['input_f']
            output_f = model.model.params['output_f']
            feature_counts = model.model.params['feature_counts']
            dropout_factor = model.model.params['dropout_factor']
            L2_factor = model.L2_factor
            model = FCN_model(input_f=input_f, output_f=output_f, feature_counts=feature_counts,
                              dropout_factor=dropout_factor, L2_factor=L2_factor,mode=mode)
            
        X_train = Xs[XY_folds[i][0]]
        X_test = Xs[XY_folds[i][1]]
        Y_train = Ys[XY_folds[i][0]]
        Y_test = Ys[XY_folds[i][1]]

        if fit_square:
            Y_train = Y_train ** 2

        if deep_learning:
            X_train_DL, X_val, Y_train_DL, Y_val = train_test_split(X_train, Y_train, test_size=0.20)
    
        if standard_scale:
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
            if deep_learning:
                X_val = scaler.transform(X_val)
            
        if deep_learning:
            _ = model.fit(X_train_DL, Y_train_DL, X_val, Y_val)
        else:
            _ = model.fit(X_train, Y_train)
    
        Y_pred = model.predict(X_test)
        if fit_square:
            Y_pred = np.sqrt(np.abs(Y_pred))
        metric_test_list.append(metric(Y_test, Y_pred))

        Y_pred = model.predict(X_train)
        if fit_square:
            Y_pred = np.sqrt(np.abs(Y_pred))
        metric_train_list.append(metric(Y_train, Y_pred))

    metric_test = np.mean(metric_test_list)
    metric_train = np.mean(metric_train_list)
    return metric_test, metric_train

In [17]:
def assign_model(model_label, Xs, cfg, mode):
    
    if model_label == 'LightGBM':
        model = model_class_dict[mode][model_label](force_col_wise=True, verbose=-1, **cfg)
    elif model_label == 'FCN':
        model = model_class_dict[mode][model_label](input_f=Xs.shape[1], output_f=1, feature_counts=[16, 16, 16, 8], mode=mode, **cfg)
    else:
        model = model_class_dict[mode][model_label](**cfg)

    return model

In [18]:
def hyperparameter_tuning(trial, Xs, Ys, mode='regressor',
                          model_label='RandomForest', n_iters=50, always_test_last_chunk=False, fit_square=False):

    deep_learning = model_label in ['FCN']
    standard_scale = not deep_learning

    if model_label in ['RandomForest', 'XGBoost', 'LightGBM']:
        cfg = {'max_depth': trial.suggest_int('max_depth', 2, 15),
               'n_estimators': trial.suggest_int('n_estimators', 10, 200)}     
    elif model_label in ['SVR', 'SVC']:
        cfg = {'C': trial.suggest_float('C', 1e-3, 2e2, log=True),
               'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])}
    elif model_label in ['NuSVR', 'NuSVC']:
        cfg = {'C': trial.suggest_float('C', 1e-3, 2e2, log=True),
               'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
               'nu': trial.suggest_float('nu', 0.1, 0.9)}
    elif model_label == 'FCN':
        cfg = {'L2_factor': trial.suggest_float('L2_factor', 1e-3, 1, log=True),
               'dropout_factor': trial.suggest_float('dropout_factor', 0, 0.5)}
    elif model_label == 'LogisticRegression':
        cfg = {'C': trial.suggest_float('C', 1e-3, 2e2, log=True),
               'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'])}
    elif model_label == 'LinearRegression':
        cfg = {}

    model = assign_model(model_label, Xs, cfg, mode)
   
    metric_list = []
    iterator = range(n_iters)
    for i in iterator:
        metric, _ = five_fold_test(Xs, Ys, model, mode=mode, 
                                   deep_learning=deep_learning, standard_scale=standard_scale,
                                   always_test_last_chunk=always_test_last_chunk, fit_square=fit_square)
        metric_list.append(metric)

    return np.mean(metric_list) - np.std(metric_list)

In [19]:
def optuna_operation(model_xcols, Y_feature, data_df, mode='regressor', speed_test=False,
                     optuna_n_trials=20, n_iters=30, always_test_last_chunk=False, fit_wind_square=False):

    if mode == 'regressor':
        metric_name = 'R2'
    elif mode == 'classifier':
        metric_name = 'F1'

    fit_square = fit_wind_square and '風速' in Y_feature
        
    model_hyperparameters_dict = {}
    model_r2_dict = {}
    
    if always_test_last_chunk:
        n_iters = 1

    model_labels = list(model_xcols.keys())
    
    for model_label in model_labels:
        X_features = model_xcols[model_label]
        Xs, Ys, _ = get_XY(data_df, Y_feature, X_features)

        this_n_iters = n_iters
        this_optuna_n_trials = optuna_n_trials

        if model_label == 'FCN':
            this_n_iters = min(this_n_iters, 1)
            if speed_test:
                this_optuna_n_trials = 4

        if model_label == 'LinearRegression':
            this_optuna_n_trials = 1
            this_n_iters = 10
            
        def target_func(trial, model_label=model_label, Xs=Xs, Ys=Ys, mode=mode,
                        n_iters=this_n_iters, always_test_last_chunk=always_test_last_chunk, fit_square=fit_square):
            return hyperparameter_tuning(trial, model_label=model_label, Xs=Xs, Ys=Ys, mode=mode,
                                         n_iters=n_iters, always_test_last_chunk=always_test_last_chunk, fit_square=fit_square)
        
        sampler = optuna.samplers.TPESampler()
        study = optuna.create_study(sampler=sampler, direction='maximize')
        with tqdm(total=this_optuna_n_trials) as pbar:
            for _ in range(this_optuna_n_trials):
                study.optimize(target_func, n_trials=1, catch=(Exception,))
                pbar.update(1)
        
        print(model_label)
        for key, v in study.best_params.items():
            print(f"Best {key} = {v}")
        print(f"Best {metric_name} = {study.best_value}")
    
        model_hyperparameters_dict[model_label] = study.best_params
        model_r2_dict[model_label] = study.best_value

    return model_hyperparameters_dict, model_r2_dict

### Ensemble

這部分的函數有：
1. cross_correlation_matrix: 由不同模型的預測誤差產生相關矩陣，僅由 get_residual_corr_matrix 呼叫。
2. sovle_optimal_weights: 由誤差相關矩陣解出最佳權重，僅由 find_optimal_weights 呼叫。
3. predict: 訓練模型並取得預測值，僅由 get_residual_corr_matrix 呼叫。
4. get_residual_corr_matrix: 計算並取得所有模型多次取樣的 Y_truth, Y_pred, 與誤差相關矩陣，僅由 find_optimal_weights 呼叫。
5. get_weighted_ensemble_metric: 輸入 get_residual_corr_matrix 的計算結果之後，得出模型評估表格，僅由 find_optimal_weights 呼叫。
6. find_optimal_weights: 統整以上函數，解出最佳權重，並印出模型評估表格。
7. save_model_metadata: 儲存這份筆記本得到的每個被預測值所採用的模型組合，以及每個模型採用的特徵、超參數與權重。

流程控制函數中會呼叫 find_optimal_weights 與 save_model_metadata

In [20]:
def cross_correlation_matrix(residuals):
    N = len(residuals)
    matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            matrix[i][j] = np.mean(np.array(residuals[i]) * np.array(residuals[j]))

    for i in range(1, N):
        for j in range(i):
            matrix[i][j] = matrix[j][i]

    return matrix

In [21]:
def sovle_optimal_weights(matrix):
    N = matrix.shape[0]
    def objective(weights):
        return weights.T @ matrix @ weights

    initial_weights = np.array([1/N] * N)
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1)] * N
    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    optimal_weights = result.x
    return optimal_weights

In [22]:
def predict(model_label, Y_train, train_ind, test_ind, mode,
            model_hyperparameters_dict, model_xcols, data_df, Y_feature, fit_wind_square=False):

    fit_square = fit_wind_square and '風速' in Y_feature
    
    if fit_square:
        Y_train = Y_train**2
    
    X_features = model_xcols[model_label]
    Xs, _, _ = get_XY(data_df, Y_feature=Y_feature, X_features=X_features)

    model = assign_model(model_label, Xs, cfg=model_hyperparameters_dict[model_label], mode=mode)

    deep_learning = False
    if model_label == 'FCN':
        deep_learning = True

    X_train = Xs[train_ind]
    X_test = Xs[test_ind]
    
    if deep_learning:
        X_train_dl, X_val, Y_train_dl, Y_val = train_test_split(X_train, Y_train, test_size=0.20)
        _ = model.fit(X_train_dl, Y_train_dl, X_val, Y_val)
    else:
        scaler = StandardScaler()
        X_scaler = scaler.fit(X_train)
        X_train = X_scaler.transform(X_train)
        X_test = X_scaler.transform(X_test)
        _ = model.fit(X_train, Y_train)
    YP = model.predict(X_test)
    if fit_square:
        YP = np.sqrt(np.abs(YP))
    return YP 

In [23]:
def get_residual_corr_matrix(model_hyperparameters_dict, ensemble_models, model_xcols,
                             data_df, Ys, Y_feature, mode,
                             n_iters, n_samples, fit_wind_square=False):
    
    def get_prediction_func(model_hyperparameters_dict=model_hyperparameters_dict,
                            model_xcols=model_xcols,
                            data_df=data_df,
                            Y_feature=Y_feature,
                            mode=mode,
                            fit_wind_square=fit_wind_square):
        def func(model_label, Y_train, train_ind, test_ind):
            return predict(model_label, Y_train, train_ind, test_ind, mode,
                           model_hyperparameters_dict, model_xcols, data_df, Y_feature, fit_wind_square=fit_wind_square)
        return func
        
    get_prediction = get_prediction_func()
    
    Y_pred_iters, Y_test_iters, model_metric = [], [], []
    matrix = np.zeros((len(ensemble_models), len(ensemble_models)))
    for i in tqdm(range(n_iters)):
        train_ind, test_ind, _, _ = train_test_split(np.arange(n_samples), np.arange(n_samples), test_size=0.2)
        
        Y_train = Ys[train_ind]
        Y_test = Ys[test_ind]
        
        Y_preds, this_metric = [], []
        for model_label in ensemble_models:
            YP = get_prediction(model_label, Y_train, train_ind, test_ind)
            if mode == 'regressor':
                this_metric.append(MAE(Y_test, YP))
            elif mode == 'classifier':
                YP[np.where(YP<0.5)] = 0
                YP[np.where(YP>=0.5)] = 1
                this_metric.append(f1_score(Y_test, YP))
            Y_preds.append(YP)
            
        residuals = Y_preds - np.array([Y_test] * len(Y_preds)).reshape(len(Y_preds),-1)
        matrix += cross_correlation_matrix(residuals)

        model_metric.append(this_metric)
        Y_pred_iters.append(Y_preds)
        Y_test_iters.append(Y_test)
    matrix = matrix / n_iters
    return matrix, model_metric, Y_pred_iters, Y_test_iters

In [24]:
def get_weighted_ensemble_metric(Y_pred_iters, Y_test_iters, weights, mode):
    n_iters = len(Y_pred_iters)
    weighted_metric = []
    for i in range(n_iters):
        weighted_YP = np.sum(Y_pred_iters[i] * np.concatenate([weights.reshape(-1,1),] * Y_test_iters[0].shape[0], axis = 1), axis=0)
        if mode == 'regressor':
            weighted_metric.append(MAE(Y_test_iters[i], weighted_YP))
        elif mode == 'classifier':
            weighted_YP[np.where(weighted_YP<0.5)] = 0
            weighted_YP[np.where(weighted_YP>=0.5)] = 1
            weighted_metric.append(f1_score(Y_test_iters[i], weighted_YP))
    weighted_metric = np.array(weighted_metric).reshape(-1, 1)
    return weighted_metric

In [25]:
def find_optimal_weights(model_hyperparameters_dict, model_xcols, 
                         data_df, Y_feature, mode='regressor', fit_wind_square=False,
                         n_iters=200, weights=None):

    if mode == 'regressor':
        metric_name = 'MAE'
    elif mode == 'classifier':
        metric_name = 'F1'       
    
    if weights is None:
        ensemble_models = list(model_hyperparameters_dict.keys())
    else:
        ensemble_models = list(weights.keys())

    n_models = len(ensemble_models)

    X_features = model_xcols[ensemble_models[0]]
    Xs, Ys, _ = get_XY(data_df, Y_feature=Y_feature, X_features=X_features)
    n_samples = Xs.shape[0]

    matrix, model_metric, Y_pred_iters, Y_test_iters = get_residual_corr_matrix(model_hyperparameters_dict=model_hyperparameters_dict,
                                                                                model_xcols=model_xcols, ensemble_models=ensemble_models,
                                                                                data_df=data_df, Ys=Ys, Y_feature=Y_feature, mode=mode,
                                                                                n_iters=n_iters, n_samples=n_samples, fit_wind_square=fit_wind_square)
    
    if weights is None:
        optimal_weights = sovle_optimal_weights(matrix)
    else:
        optimal_weights = weights

    uniform_weights = np.array([1/n_models] * n_models)
    uniform_metric = get_weighted_ensemble_metric(Y_pred_iters, Y_test_iters, uniform_weights, mode)
    optimal_metric = get_weighted_ensemble_metric(Y_pred_iters, Y_test_iters, optimal_weights, mode)

    array_metric = np.concatenate([model_metric, uniform_metric, optimal_metric], axis=1)
    
    metric_dict = {
        'Model': ensemble_models + ['Ensemble', 'Weighted_Ensemble'],
        f'Avg {metric_name}': list(np.mean(array_metric, axis=0)), 
        f'Std {metric_name}': list(np.std(array_metric, axis=0)),
        '90th percentile': list(np.sort(array_metric, axis=0)[int(array_metric.shape[0] * 0.9) - 1]),
        '10th percentile': list(np.sort(array_metric, axis=0)[int(array_metric.shape[0] * 0.1) - 1])
        }
    
    df = pd.DataFrame(metric_dict)
    if mode == 'regressor':
        df = df.sort_values('90th percentile').reset_index(drop=True)
    elif mode == 'classifier':
        df = df.sort_values('10th percentile', ascending=False).reset_index(drop=True)

    if weights is not None:
        return df
        
    optimal_weights_dict = {ensemble_models[i]: w for i, w in enumerate(optimal_weights)}
        
    return df, optimal_weights_dict

In [26]:
def save_model_metadata(file_path, model_xcols, model_hyperparameters_dict, optimal_weights, fit_wind_square=False):
    model_labels = list(model_hyperparameters_dict)
    output_dict = {
        'X_feature_dict':{},
        'hyperparameters_dict':{},
        'weights':{}
    }
    for model_label in model_labels:
        if optimal_weights[model_label] > 0.0005:
            output_dict['X_feature_dict'][model_label] = model_xcols[model_label]
            output_dict['hyperparameters_dict'][model_label] = model_hyperparameters_dict[model_label]
            output_dict['weights'][model_label] = optimal_weights[model_label]
            output_dict['fit_wind_square'] = fit_wind_square

    with open(file_path, 'w') as f:
        json.dump(output_dict, f)

### 流程控制

主要被主程式呼叫的函數  
負責管理超參數及權重的計算與存取

In [27]:
def flow_control(Y_feature, model_xcols, data_df, mode='regressor', speed_test=False, fit_wind_square=False,
                 train_model_path=train_model_path, optuna_has_done=optuna_has_done, weights_has_determined=weights_has_determined, run=False):

    n_iter_dict = {
        'hyper_parameter': 15,
        'ensemble_weight': 200
    }
    if speed_test:
        n_iter_dict = {
            'hyper_parameter': 1,
            'ensemble_weight': 20
        }
    
    this_model_path = f'{train_model_path}{Y_feature}/'
    os.makedirs(this_model_path, exist_ok=True)

    # 如果指定的 meta 檔存在，並且初始參數規定不須重新計算，則套用存檔數值。
    if os.path.exists(f'{train_model_path}{Y_feature}/meta.json') and not run:
        with open(f'{train_model_path}{Y_feature}/meta.json', 'r') as f:
            meta = json.load(f)
    else:
        optuna_has_done[Y_feature] = False
        weights_has_determined[Y_feature] = False

    # 超參數  
    if optuna_has_done[Y_feature]:
        model_xcols = meta['X_feature_dict']
        model_hyperparameters_dict = meta['hyperparameters_dict']
    else: 
        print('Start to tune hyperparameters')
        model_hyperparameters_dict, model_r2_dict = optuna_operation(model_xcols, Y_feature, data_df, mode=mode,
                                                                     n_iters=n_iter_dict['hyper_parameter'],
                                                                     speed_test=speed_test, fit_wind_square=fit_wind_square)
    
    # 集成權重
    if weights_has_determined[Y_feature]:
        optimal_weights = meta['weights']
        df = pd.read_csv(f'{this_model_path}predict_MAE.csv')
        display(df)
        print('Weights:')
        for i, k in enumerate(model_hyperparameters_dict.keys()):
            print(f'{k}: {optimal_weights[k]:.3f}')
    else:
        print('Start to determine Ensemble weights.')
        if 'FCN' in model_hyperparameters_dict.keys():
            n_iters = int(n_iter_dict['ensemble_weight']/4)
        else:
            n_iters = n_iter_dict['ensemble_weight']
        df, optimal_weights = find_optimal_weights(model_hyperparameters_dict, model_xcols, data_df,
                                                   Y_feature=Y_feature, mode=mode, n_iters=n_iters, fit_wind_square=fit_wind_square)
        print(Y_feature)
        display(df)
        df.to_csv(f'{this_model_path}predict_MAE.csv', index=False, encoding='utf-8-sig')
        print('Weights:')
        for i, k in enumerate(model_hyperparameters_dict.keys()):
            print(f'{k}: {optimal_weights[k]:.3f}')

    if not (weights_has_determined[Y_feature] and optuna_has_done[Y_feature]):
        print(' ')
        print(' ')
        print('**Copy and Paste following lines into the next cell.**')
        for model_label in model_hyperparameters_dict.keys():
            print('##### ' + model_label)
            for key, v in model_hyperparameters_dict[model_label].items():
                print(f"Best {key} = {v}  ")
            if 'model_r2_dict' in locals().keys():
                print(f"Best R2 = {model_r2_dict[model_label]}  ")
            print(f'Weight = {optimal_weights[model_label]:.3f}')
        print(' ')
        print(' ')
        save_model_metadata(this_model_path + 'meta.json', model_xcols, model_hyperparameters_dict, optimal_weights, fit_wind_square=fit_wind_square)

## 預測氣象數值

在先前對於如何用氣象資料預測電力資料的探索，我們發現中央氣象署提供的歷史氣象資料中，以每天的最高氣溫、最低氣溫、平均氣溫、風速與全天空日射量等五個數值比較重要  
同時從中央氣象署網站下載的氣象預報當中，可以找到每個鄉鎮市區每三個小時的天氣狀況、氣溫、風速、風向、相對溼度等資訊  
這邊的天氣狀況就是晴、多雲、短暫陣雨之類的文字敘述，根據我的觀察，這樣的文字敘述可以歸類為七種：  
晴、多雲、陰、短暫陣雨、短暫陣雨或雷雨、午後短暫雷陣雨、陣雨或雷雨
在我的資料表中預先將這七種預報文字進行 one-hot coding 處理

我判斷這樣的文字敘述跟日照率，也就是實測日照時數與天文日照時數之比率比較相關，所以這邊預測的氣象數值鎖定在以下五個值：  
日照率、最高氣溫、最低氣溫、(平均)氣溫、風速  

而預測的樣本單位則是每天每氣象站算是一個樣本，而考慮的氣象站為：  
臺北站、高雄站、嘉義站、東吉島站、臺中電廠站、臺西站等六站

另外在每個預測標的下方的結果表格中，為了瞭解每個模型以及每種集成學習方式分別的預測誤差，以及誤差值的穩定性  
所以我將 MAE 的平均值、標準差、第 10 與第 90 百分位都列出來  
表格是按誤差的第 90 百分位排序的，以觀察各預測方式的穩定性
除了夜尖峰是預測 Yes or No 問題，屬於分類問題而非回歸問題  
所以表格呈現的是 f1-score，也相應的改成按照第 10 百分位排序

表格中的 Ensemble 代表每個模型權重一致的集成學習  
而 Weighted_Ensemble 則代表使用從相關矩陣中解出的最佳權重的集成學習

### 時段風速與氣溫

這邊的時段風速與氣溫，指的是三個時段：午後 (12-15點)、下午 (15-18點)、傍晚(18-21點) 的平均氣溫與風速  
由於用電尖峰通常發生在這三個時段，所以後面的風力發電與夜尖峰的預測會用到這些數據  
氣溫單位為攝氏度，風速單位則為公尺每秒  

In [29]:
for des in ['午後', '下午', '傍晚']:
    model_xcols = {
        'LinearRegression': ['溫度', 'Town'],
        'SVR': ['溫度', 'Town'],
        'RandomForest': ['溫度', 'Town'],
        'XGBoost': ['溫度', 'Town'],
        'LightGBM': ['溫度', 'Town'],
        'NuSVR': ['溫度', 'Town'],
        'FCN': ['溫度', 'Town'],
    }
    Y_feature = f'{des}平均氣溫'
    flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test)

    model_xcols = {
        'LinearRegression': ['風速', '風速超過上限', 'Town'],
        'SVR': ['風速', '風速超過上限', 'Town'], 
        'RandomForest': ['風速', '風速超過上限', 'Town'],
        'XGBoost': ['風速', '風速超過上限', 'Town'],
        'LightGBM': ['風速', '風速超過上限', 'Town'],
        'NuSVR': ['風速', '風速超過上限', 'Town'],
        'FCN': ['風速', '風速超過上限', 'Town'],
    }
    Y_feature = f'{des}平均風速'
    flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test, fit_wind_square=True)
         

,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Ensemble,1.016024,0.077544,1.094937,0.915762
1,NuSVR,1.017883,0.077631,1.102928,0.908307
2,Weighted_Ensemble,1.013767,0.077928,1.103011,0.906017
3,SVR,1.018647,0.084349,1.107473,0.900105
4,LinearRegression,1.029775,0.077656,1.118068,0.924942
5,RandomForest,1.072024,0.076958,1.136645,0.966919
6,LightGBM,1.051730,0.082741,1.163560,0.959929
7,FCN,1.069178,0.079540,1.166062,0.953253
8,XGBoost,1.079415,0.077371,1.188670,0.973755


Weights:
SVR: 0.065
RandomForest: 0.036
XGBoost: 0.162
LightGBM: 0.149
NuSVR: 0.568
FCN: 0.021


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,0.996972,0.099125,1.144687,0.860772
1,Ensemble,1.016579,0.099557,1.151503,0.876464
2,NuSVR,1.040528,0.101580,1.168735,0.895713
3,SVR,1.032422,0.103455,1.179626,0.889871
4,RandomForest,1.048379,0.101433,1.181707,0.883286
5,XGBoost,1.091584,0.109815,1.239234,0.934853
6,LightGBM,1.143701,0.104702,1.264282,0.999941
7,FCN,1.132761,0.118036,1.287907,0.990246
8,LinearRegression,1.428423,0.116999,1.593565,1.250454


Weights:
LinearRegression: 0.044
SVR: 0.080
RandomForest: 0.191
XGBoost: 0.156
LightGBM: 0.060
NuSVR: 0.406
FCN: 0.063


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,SVR,1.151137,0.094567,1.236929,1.023380
1,Ensemble,1.167160,0.089075,1.261887,1.073191
2,Weighted_Ensemble,1.165917,0.087976,1.261940,1.076796
3,NuSVR,1.185907,0.082394,1.269547,1.086835
4,FCN,1.193208,0.089708,1.292541,1.081001
5,LinearRegression,1.184456,0.086118,1.295467,1.081082
6,RandomForest,1.223489,0.094562,1.319498,1.095391
7,LightGBM,1.206531,0.091597,1.324407,1.092474
8,XGBoost,1.201702,0.091231,1.330249,1.089797


...

Weights:
LinearRegression: 0.016
SVR: 0.427
RandomForest: 0.190
XGBoost: 0.256
NuSVR: 0.091
FCN: 0.019


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,NuSVR,0.973880,0.066114,1.056402,0.871490
1,Weighted_Ensemble,0.972884,0.064860,1.060648,0.874643
2,Ensemble,0.984360,0.066095,1.063476,0.877694
3,SVR,0.974617,0.068115,1.066731,0.862826
4,LinearRegression,1.000556,0.063575,1.093493,0.918447
5,FCN,1.021564,0.073064,1.113393,0.925463
6,RandomForest,1.038679,0.068475,1.119132,0.935224
7,XGBoost,1.046935,0.066893,1.129241,0.955047
8,LightGBM,1.035971,0.072832,1.133903,0.925187


Weights:
LinearRegression: 0.383
RandomForest: 0.082
LightGBM: 0.066
NuSVR: 0.425
FCN: 0.045


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,1.025952,0.106684,1.178086,0.881694
1,SVR,1.072056,0.099584,1.195407,0.935462
2,RandomForest,1.075123,0.119593,1.232388,0.914874
3,Ensemble,1.081856,0.115877,1.241232,0.922749
4,NuSVR,1.157279,0.116121,1.295116,0.994493
5,XGBoost,1.129415,0.134320,1.304509,0.928575
6,LightGBM,1.226312,0.138586,1.383665,1.037181
7,FCN,1.256510,0.172929,1.422130,1.017980
8,LinearRegression,1.519090,0.133068,1.691518,1.332361


Weights:
SVR: 0.458
RandomForest: 0.218
XGBoost: 0.234
NuSVR: 0.013
FCN: 0.077


### 日照率

雖然實際上跟太陽能發電量比較相關的是全天空日射量  
但是在 EDA 環節我們可以看到，日照率乘上天文日射量之後，跟全天空日射量有 r=0.9 左右的極高相關性  
而天文日射量是給定日期就可以確切計算出來的  
所以這邊選擇日照率為預測標的  
單位為百分點  

In [30]:
# 被預測的標的
Y_feature = '日照率'
# 定義集成學習使用的模型以及模型們各自使用的 X 特徵
model_xcols = {
        'RandomForest': ['晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', 'Town'],
        'XGBoost': ['晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', 'Town'],
        'LightGBM': ['晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', 'Town'],
        'SVR': ['晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', 'Town'],
        'NuSVR': ['晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', 'Town'],
        'FCN': ['晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', 'Town'],
    }

flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test)

,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,15.486858,0.911381,16.724693,14.432103
1,Ensemble,15.638591,0.949343,16.892975,14.642680
2,NuSVR,15.687880,0.934292,17.137901,14.470914
3,SVR,15.849576,0.935820,17.145180,14.702553
4,XGBoost,16.018460,1.105321,17.545488,14.658275
5,LightGBM,16.378106,1.032178,17.843652,15.199501
6,RandomForest,16.330463,1.103974,17.953743,15.195999
7,FCN,17.160312,1.591835,18.757145,15.330530


Weights:
XGBoost: 0.109
SVR: 0.211
NuSVR: 0.593
FCN: 0.087


### 高溫

In [31]:
Y_feature = '最高氣溫'
model_xcols = {
    'LinearRegression': ['溫度', 'Town'],
    'FCN': ['溫度', 'Town'],
    'RandomForest': ['溫度', 'Town'],
    'XGBoost': ['溫度', 'Town'],
    'SVR': ['溫度', 'Town'],
    'NuSVR': ['溫度', 'Town'],
    'LightGBM': ['溫度', 'Town'],
}

flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test)

,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,0.836637,0.064244,0.920384,0.756974
1,Ensemble,0.840220,0.064511,0.926768,0.761985
2,NuSVR,0.858278,0.063770,0.948722,0.775337
3,LinearRegression,0.872909,0.059748,0.952706,0.789783
4,LightGBM,0.862172,0.065117,0.952722,0.778038
5,SVR,0.863002,0.063617,0.954467,0.764740
6,XGBoost,0.880655,0.067135,0.970396,0.782551
7,FCN,0.883994,0.068482,0.975357,0.798511
8,RandomForest,0.898266,0.069961,0.995361,0.802322


Weights:
FCN: 0.019
XGBoost: 0.126
SVR: 0.254
NuSVR: 0.396
LightGBM: 0.205


### 低溫

In [32]:
Y_feature = '最低氣溫'
model_xcols = {
    'LinearRegression': ['溫度', 'Town'],
    'FCN': ['溫度', 'Town'],
    'RandomForest': ['溫度', 'Town'],
    'XGBoost': ['溫度', 'Town'],
    'SVR': ['溫度', 'Town'],
    'NuSVR': ['溫度', 'Town'],
    'LightGBM': ['溫度', 'Town'],
}

flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test)

,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,0.750792,0.046190,0.805984,0.701593
1,Ensemble,0.756933,0.043558,0.809994,0.706888
2,SVR,0.758514,0.047002,0.819864,0.705170
3,NuSVR,0.757797,0.049196,0.823244,0.704429
4,LinearRegression,0.777768,0.042612,0.823987,0.719275
5,XGBoost,0.782573,0.040923,0.828479,0.729008
6,LightGBM,0.784967,0.044573,0.829320,0.724568
7,FCN,0.773980,0.048109,0.832870,0.700707
8,RandomForest,0.789911,0.042844,0.833785,0.739701


Weights:
LinearRegression: 0.116
FCN: 0.088
RandomForest: 0.123
SVR: 0.217
NuSVR: 0.456


### 平均溫

In [33]:
Y_feature = '氣溫'
model_xcols = {
    'LinearRegression': ['溫度', 'Town'],
    'FCN': ['溫度', 'Town'],
    'RandomForest': ['溫度', 'Town'],
    'XGBoost': ['溫度', 'Town'],
    'SVR': ['溫度', 'Town'],
    'NuSVR': ['溫度', 'Town'],
    'LightGBM': ['溫度', 'Town'],
}

flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test)

,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,0.629872,0.043186,0.689697,0.580058
1,SVR,0.637682,0.044794,0.700648,0.591264
2,Ensemble,0.639699,0.040776,0.701200,0.586371
3,LightGBM,0.667188,0.039534,0.723855,0.615744
4,LinearRegression,0.668558,0.044715,0.725722,0.606610
5,NuSVR,0.668425,0.043992,0.727412,0.613019
6,XGBoost,0.667720,0.041845,0.734836,0.608941
7,RandomForest,0.678242,0.036658,0.735712,0.629860
8,FCN,0.682389,0.046247,0.742668,0.619697


Weights:
XGBoost: 0.243
SVR: 0.408
NuSVR: 0.295
LightGBM: 0.054


### 風速

In [34]:
Y_feature = '風速'
model_xcols = {
    'FCN': ['風速', '東西風', '南北風', '溫度', '風速超過上限', 'Town'],
    'RandomForest': ['風速', '東西風', '南北風', '晴', '多雲', '陰', '短暫陣雨', '短暫陣雨或雷雨', '午後短暫雷陣雨', '陣雨或雷雨', '相對溼度', '溫度', '風速超過上限', 'Town'],
    'XGBoost': ['風速', '東西風', '南北風', '溫度', '風速超過上限', 'Town'],
    'LightGBM': ['風速', '東西風', '南北風', '溫度', '風速超過上限', 'Town'],
    'SVR': ['風速', '東西風', '南北風', '溫度', '風速超過上限', 'Town'],
    'NuSVR': ['風速', '東西風', '南北風', '溫度', '風速超過上限', 'Town'],
}

flow_control(Y_feature, model_xcols, forecast_obs_df, speed_test=speed_test, fit_wind_square=True)

,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,0.719321,0.073809,0.823198,0.621993
1,Ensemble,0.708907,0.080031,0.833734,0.620741
2,RandomForest,0.791804,0.090215,0.919398,0.690969
3,SVR,0.774478,0.104652,0.925111,0.642194
4,NuSVR,0.792389,0.102551,0.949080,0.653803
5,XGBoost,0.818664,0.093935,0.954881,0.695778
6,LightGBM,0.900057,0.088054,1.014452,0.790349
7,FCN,0.893177,0.111486,1.033851,0.733650


Weights:
FCN: 0.438
RandomForest: 0.087
XGBoost: 0.158
LightGBM: 0.001
SVR: 0.317


##### FCN
Best L2_factor = 0.012867906181328041  
Best dropout_factor = 0.010144684642082125  
Best R2 = 0.3546731054062476  
Weight = 0.055
##### RandomForest
Best max_depth = 10  
Best n_estimators = 85  
Best R2 = 0.3786591592946854  
Weight = 0.214
##### XGBoost
Best max_depth = 4  
Best n_estimators = 149  
Best R2 = 0.31422596898636684  
Weight = 0.239
##### LightGBM
Best max_depth = 3  
Best n_estimators = 21  
Best R2 = 0.3483545440894781  
Weight = 0.000
##### SVR
Best C = 0.610471421488569  
Best kernel = linear  
Best R2 = 0.4862990720822355  
Weight = 0.472
##### NuSVR
Best C = 0.033558469259646846  
Best kernel = linear  
Best nu = 0.4716071317344285  
Best R2 = 0.47560151954235624  
Weight = 0.019

## 預測電力資料

台電官網上可以抓到的歷史電力資料，據我後來觀察，應該是每天用電負載尖峰的那一刻，每個機組的發電功率  
所以我們在這邊的預測標的也會是這個數值

以下所有電力相關數字單位皆為萬瓩，1萬瓩 = 10MW = 10,000,000 W

Note: 這邊是以實時氣象觀測資料預測電力資料，但是這兩組數據基本上是同時得知的  
所以實務上我們是用氣象預報預測第二天的氣象觀測，再用這個預測值預測第二天的電力資料  
因此實際上的預測誤差會比這邊顯示的數值高一點  
實際的預測情形可以到 <a href='http://ec2-54-206-30-159.ap-southeast-2.compute.amazonaws.com:8501'> 這個網站 </a> 查看

### 風力

在我蒐集的資料範圍中，風力發電數值的標準差約為 68萬瓩

In [35]:
Y_feature = '風力'

model_xcols = {
    'LinearRegression': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
    'FCN': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
    'RandomForest': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
    'XGBoost': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
    'LightGBM': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
    'SVR': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
    'NuSVR': ['風速', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '午後平均風速', '下午平均風速', '傍晚平均風速', '風力_90天內最大值'],
}

flow_control(Y_feature, model_xcols, weather_power_df, speed_test=speed_test)

Start to tune hyperparameters


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.55it/s]


LinearRegression
Best R2 = 0.6388840599150699


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [37:25<00:00, 112.29s/it]


FCN
Best L2_factor = 0.01607171548009282
Best dropout_factor = 0.1230711635938731
Best R2 = 0.804103048583664


...

NuSVR
Best C = 22.350475289282098
Best kernel = linear
Best nu = 0.437596140921024
Best R2 = 0.6379833978278489
Start to determine Ensemble weights.


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [22:02<00:00, 26.46s/it]

風力


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,17.131029,1.387935,19.287405,15.251083
1,SVR,18.247416,1.507956,20.054051,15.973281
2,RandomForest,18.449012,1.495551,20.541925,16.460292
3,Ensemble,19.161804,1.379464,20.756518,17.340299
4,LightGBM,18.791344,1.545246,20.871922,16.740362
5,XGBoost,19.311639,1.595563,21.490941,17.071105
6,FCN,20.354390,1.620722,22.255472,17.934310
7,NuSVR,29.483480,1.852564,31.876666,27.050153
8,LinearRegression,29.758345,1.730010,31.943787,27.318390


Weights:
LinearRegression: 0.000
FCN: 0.033
RandomForest: 0.342
XGBoost: 0.064
LightGBM: 0.068
SVR: 0.493
NuSVR: 0.000
 
 
**Copy and Paste following lines into the next cell.**
##### LinearRegression
Best R2 = 0.6388840599150699  
Weight = 0.000
##### FCN
Best L2_factor = 0.01607171548009282  
Best dropout_factor = 0.1230711635938731  
Best R2 = 0.804103048583664  
Weight = 0.033
##### RandomForest
Best max_depth = 13  
Best n_estimators = 166  
Best R2 = 0.8282167151617582  
Weight = 0.342
##### XGBoost
Best max_depth = 4  
Best n_estimators = 39  
Best R2 = 0.8101959890570126  
Weight = 0.064
##### LightGBM
Best max_depth = 4  
Best n_estimators = 110  
Best R2 = 0.8236617322685107  
Weight = 0.068
##### SVR
Best C = 90.39459641694827  
Best kernel = rbf  
Best R2 = 0.8404929286098619  
Weight = 0.493
##### NuSVR
Best C = 22.350475289282098  
Best kernel = linear  
Best nu = 0.437596140921024  
Best R2 = 0.6379833978278489  
Weight = 0.000
 
 


### 太陽能


太陽能發電數值的原始標準差約為 260 萬瓩

In [36]:
Y_feature = '太陽能'

model_xcols = {
    'LinearRegression': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
    'FCN': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
    'RandomForest': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
    'XGBoost': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
    'LightGBM': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
    'SVR': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
    'NuSVR': ['氣溫', '最高氣溫', '最低氣溫', '全天空日射量', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '太陽能_90天內最大值'],
}

flow_control(Y_feature, model_xcols, weather_power_df, speed_test=speed_test)

Start to tune hyperparameters


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.43it/s]


LinearRegression
Best R2 = 0.7130585250779067


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [42:22<00:00, 127.14s/it]


FCN
Best L2_factor = 0.019126819866377914
Best dropout_factor = 0.20144382293032162
Best R2 = 0.7937561628618668


...

NuSVR
Best C = 65.15154286843989
Best kernel = linear
Best nu = 0.31732690348091874
Best R2 = 0.7136687960749358
Start to determine Ensemble weights.


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [22:28<00:00, 26.97s/it]

太陽能


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,82.589763,8.562348,91.441819,70.832498
1,FCN,82.907245,9.598564,94.835253,68.426091
2,Ensemble,88.348120,8.412893,96.392795,75.015392
3,XGBoost,90.254223,8.094503,97.081054,80.428434
4,LightGBM,89.471958,7.719940,97.412563,80.381000
5,RandomForest,87.321462,8.719561,98.598250,75.492007
6,NuSVR,107.920873,8.376351,118.034949,96.303985
7,SVR,106.546204,8.843818,119.356014,94.055653
8,LinearRegression,108.341871,8.869337,119.880854,95.381866


Weights:
LinearRegression: 0.148
FCN: 0.304
RandomForest: 0.165
XGBoost: 0.101
LightGBM: 0.282
SVR: 0.000
NuSVR: 0.000
 
 
**Copy and Paste following lines into the next cell.**
##### LinearRegression
Best R2 = 0.7130585250779067  
Weight = 0.148
##### FCN
Best L2_factor = 0.019126819866377914  
Best dropout_factor = 0.20144382293032162  
Best R2 = 0.7937561628618668  
Weight = 0.304
##### RandomForest
Best max_depth = 12  
Best n_estimators = 186  
Best R2 = 0.755965324968995  
Weight = 0.165
##### XGBoost
Best max_depth = 4  
Best n_estimators = 153  
Best R2 = 0.731714134991313  
Weight = 0.101
##### LightGBM
Best max_depth = 4  
Best n_estimators = 79  
Best R2 = 0.7624372430071734  
Weight = 0.282
##### SVR
Best C = 55.451103794310384  
Best kernel = linear  
Best R2 = 0.7059128451750041  
Weight = 0.000
##### NuSVR
Best C = 65.15154286843989  
Best kernel = linear  
Best nu = 0.31732690348091874  
Best R2 = 0.7136687960749358  
Weight = 0.000
 
 


### 尖峰負載

尖峰負載的原始標準差約為 410 萬瓩

In [37]:
Y_feature = '尖峰負載'

model_xcols = {
    'LinearRegression': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
    'FCN': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
    'RandomForest': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
    'XGBoost': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
    'LightGBM': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
    'SVR': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
    'NuSVR': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '1~3月', '11~12月', '尖峰負載_90天內最大值'],
}

flow_control(Y_feature, model_xcols, weather_power_df, speed_test=speed_test)

Start to tune hyperparameters


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.81it/s]


LinearRegression
Best R2 = 0.8932133341950986


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [40:05<00:00, 120.25s/it]


FCN
Best L2_factor = 0.05143657165118318
Best dropout_factor = 0.04708143758424782
Best R2 = 0.9574404132918009


...

NuSVR
Best C = 44.789414155205755
Best kernel = rbf
Best nu = 0.4438169636077629
Best R2 = 0.9409780740683609
Start to determine Ensemble weights.


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [22:01<00:00, 26.43s/it]

尖峰負載


,Model,Avg MAE,Std MAE,90th percentile,10th percentile
0,Weighted_Ensemble,57.050633,4.819045,63.492614,51.843071
1,FCN,62.070919,5.351254,69.740382,55.713803
2,Ensemble,65.719759,5.018497,72.890579,59.429290
3,LightGBM,68.549678,5.308991,74.379567,61.760590
4,RandomForest,68.347069,6.173471,76.003162,60.038906
5,XGBoost,68.292020,6.076207,76.892150,60.906355
6,NuSVR,75.620199,6.166200,82.857346,66.146758
7,LinearRegression,108.626276,6.366205,115.286354,100.222042
8,SVR,108.561072,7.109422,117.415973,98.903691


Weights:
LinearRegression: 0.000
FCN: 0.516
RandomForest: 0.000
XGBoost: 0.257
LightGBM: 0.112
SVR: 0.000
NuSVR: 0.115
 
 
**Copy and Paste following lines into the next cell.**
##### LinearRegression
Best R2 = 0.8932133341950986  
Weight = 0.000
##### FCN
Best L2_factor = 0.05143657165118318  
Best dropout_factor = 0.04708143758424782  
Best R2 = 0.9574404132918009  
Weight = 0.516
##### RandomForest
Best max_depth = 10  
Best n_estimators = 94  
Best R2 = 0.9453433482769796  
Weight = 0.000
##### XGBoost
Best max_depth = 4  
Best n_estimators = 117  
Best R2 = 0.9475018773568223  
Weight = 0.257
##### LightGBM
Best max_depth = 3  
Best n_estimators = 194  
Best R2 = 0.945484763554383  
Weight = 0.112
##### SVR
Best C = 182.01031669105635  
Best kernel = linear  
Best R2 = 0.8917291339706976  
Weight = 0.000
##### NuSVR
Best C = 44.789414155205755  
Best kernel = rbf  
Best nu = 0.4438169636077629  
Best R2 = 0.9409780740683609  
Weight = 0.115
 
 


### 夜尖峰

通常台灣全天用電的峰值會發生在下午 1 到 2 點，但是在非工作日或是氣溫較低的時候，有時用電峰值會發生在傍晚 5~7 點左右  
這種狀況之下，台電的歷史資料中太陽能的部分就會變成 0 或者很接近 0，因為取樣時太陽快要或已經下山  
為了處理「夜尖峰」狀況對於太陽能數值預測的影響，我也嘗試預測了夜尖峰的發生與否  
這個問題跟前面預測數值的回歸問題不同，基本上是個分類問題，所以衡量指標變成了 f1-score

In [38]:
Y_feature = '夜尖峰'
model_xcols = {
    'FCN': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '午後平均氣溫', '下午平均氣溫', '傍晚平均氣溫'],
    'LogisticRegression': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '午後平均氣溫', '下午平均氣溫', '傍晚平均氣溫'],
    'RandomForest': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '午後平均氣溫', '下午平均氣溫', '傍晚平均氣溫'],
    'XGBoost': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '午後平均氣溫', '下午平均氣溫', '傍晚平均氣溫'],
    'LightGBM': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '午後平均氣溫', '下午平均氣溫', '傍晚平均氣溫'],
    'SVC': ['氣溫', '最高氣溫', '最低氣溫', '日期數字', '假日', '週六', '週日', '補班', '白日長度', '午後平均氣溫', '下午平均氣溫', '傍晚平均氣溫']
}

flow_control(Y_feature, model_xcols, weather_power_df, mode='classifier', speed_test=speed_test)

,Model,Avg F1,Std F1,90th percentile,10th percentile
0,Ensemble,0.873267,0.033555,0.914286,0.833333
1,Weighted_Ensemble,0.878996,0.039992,0.935484,0.823529
2,LogisticRegression,0.876544,0.042070,0.931034,0.821429
3,SVC,0.878901,0.040221,0.935484,0.816327
4,XGBoost,0.859695,0.039071,0.904762,0.812500
5,FCN,0.853109,0.042431,0.909091,0.793103
6,LightGBM,0.854182,0.047632,0.914286,0.781250
7,RandomForest,0.787793,0.061470,0.843750,0.716981


Weights:
FCN: 0.144
LogisticRegression: 0.138
RandomForest: 0.167
XGBoost: 0.182
LightGBM: 0.001
SVC: 0.367
